# FSDS24 - Week 1, Lab 3: Merging and reshaping data

In this lab, we will be including two files from the Movie Stack Exchange. The original `'movie_stack_df.feather"`, but now also `'movie_stack_df_users.feather'`. Depending on your choice of approach to questions 1-3 you may not need the Users database. Both should be available on Canvas.

To remind, you can see the schema for this data at the following URL: 
https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

This lab will also be the first formative exercise to be submitted for feedback. The details about the deadline will be in the 'assignment' on Canvas. Provisionally, this is due on Wednesday at 5pm. This allows for multiple sessions with the TAs in order to explore and understand your answer to this question. 

In this lab you will want to merge the posts data in with users data in such a way that you will be able to make some statements about the users that you would not be able to otherwise. Depending on your skill set with managing data, you may want to scale your ambitions accordingly. However, you should be able with some assistance from the web, chat agents, and your peers to draft your own code in order to make some meaningful claims.  

For this exercise you will want to display skills in merging, filtering, and aggregation as well as give due regard to operationalisation. We will introduce time series data next week so we should try to consider approaches that do not rely heavily on time series operations. That said, you should be able to filter the data by datetime with little difficulty should you want to explore relationships over time. 

# Exercise 1. Considering units of analysis: Posts vs Users 

In the posts table, each post is a unit of analysis; i.e. a row in our data. Yet, the posts were made by user accounts. We can ask questions of both, either independently or together, where appropriate. In this first exercise we want you to describe a research question that focuses on specific units of analysis.

**First** construct a research question (that can be considered with this data) where our unit of analysis is the post. 

Some examples: 
- Comparative: Which is longer: a post or an answer?  
- Correlational: Do posts with more words lead to answers with more words? 

**Second** construct a research question where the unit of analysis is the user. 

Some examples:
- Comparative: Do users with a website URL answer posts more often?
- Correlational: Do users who write many posts also answer many posts?



## Answer 1. 

RQ1. Do posts with higher emotional arousal have more (or longer) comments?

RQ2. Do users with longer "About Me" sections use more egocentric speech in their posts?

### Evaluator's comments 
 > These will be terse and focus on whether the unit of analysis is correct and whether the other elements make sense as something to be explored in this data.

^^^ 

# Question 2. Operationalisation

For only one of these two research questions describe how you will operationalise your concepts.

Consider: 
- Inclusion / exclusion criteria;
- How you will establish your unit of analysis derived from the data. For example, for 'longer' posts does this include URLs? Is it by words or by characters? Would that make a difference worth articulating?
- What sort of approach might help you establish a statistical difference?


## Answer 2. 

I'm interested in comparing the user's "About Me" section to their use of egocentric
speech. An intuitve hypothesis would be that users willing to craft long bios would
exhibit more egocentric speech, focusing more on their own opinions and reactions when
posting. So, variables measuring these two properties may be correlated.

- My unit of analysis is the individaul user account.
- I will drop the "Community" user, which is a bot, and so shouldn't behave like a normal user
  on the platform.
- I will operationalize the size of "About Me" section as the number of words in the
  section's text, omitting links and HTML attributes. Users with empty "About Me"s will
  be recorded as having 0 words.
- I will operationalize the use of egocentric language as the proportion of words in
  each post's body that are first-person pronouns ("I", "me", "myself", "we", "our",
  etc.). Again, I will use the body text, omitting HTML attributes. Normalizing using
  the proportion of words, rather than just the raw count of first-person pronouns, may
  help us avoid concluding a positive correlation just because certain users are more
  verbose, both in "About Me" and in their posting.
- I will consider only question and answer posts, omitting post types like "Moderator
  nomination"s and "Wiki placeholder"s. I will compute the proportion of
  first-person pronouns for each post, then average this result for each user to perform
  a user-level analysis.

### Evaluator's comments 
 > These will be terse and focus on whether the details used to clarify the operationalisation are clear enough that one can see how this could be accomplished with the SE data.

 ^^^

# Question 3. Performing an operationalisation 

You are welcome to simply answer the research question if you are curious, but here we have a more modest goal: perform an operationalisation on the data. That is, take one of the proposed measures and get the data in a form where you can provide descriptive statistics about that concept as a variable. For example, for word length you would first count the words per post, which would be either its own column in the DataFrame or its own Series and then plot the distribution of word length. Document any meaningful steps you had to take. In the word length case, we might document how we split the posts into words. 

In [1]:
import pandas as pd

In [2]:
# Read in the users and posts data to DataFrames
users_df = pd.read_feather('data/movies_stack_df_users.feather')
posts_df = pd.read_feather('data/movies_stack_df_posts.feather')

In [3]:
# Add labels for each post type, mapped from PostTypeId
# This code is copied exactly from the Week01_1_VW_Distributions notebook
new_labels = {
    '1': "Question",
    '2': "Answer", 
    '3': "Orphaned tag wiki",
    '4': "Tag wiki excerpt",
    '5': "Tag wiki",
    '6': "Moderator nomination", 
    '7': "Wiki placeholder"
}

posts_df["PostTypeLabel"] = posts_df["PostTypeId"].map(new_labels)

I want to keep only question and answer posts, so let's remove the rest.

In [4]:
print(f"{len(posts_df)} posts total.")
posts_df = posts_df[posts_df["PostTypeLabel"].isin(["Question", "Answer"])]
print(f"{len(posts_df)} question and answer posts.")

64054 posts total.
59606 question and answer posts.


I also need to drop the 'Community' user, which is a bot and so is not representative of
the Stack Exchange user base.

In [5]:
print(f"{len(users_df)} users before...")
users_df = users_df[users_df["Id"] != "-1"]
print(f"{len(users_df)} users after.")

73905 users before...
73904 users after.


Now, I'll merge posts with their users, adding suffixes for colliding columns. This
results in a DataFrame at the level of individual posts, which is fine for now—eventually we'll do a `groupby` to get back to the user level.

In [6]:
posts_with_users_df = posts_df.merge(users_df, left_on='OwnerUserId', right_on='Id', suffixes=("_post", "_user"))

Here, I'll create a column for the length of body text in the "About Me" section. The
`AboutMe` column contains HTML attributes, so I'll have to remove those before splitting
and counting the words. I use `BeautifulSoup` to do this.

In [7]:
from typing import Optional
from bs4 import BeautifulSoup

def get_html_text(html: Optional[str]) -> str:
    if html: return BeautifulSoup(html).get_text()

    # Return empty if the original `AboutMe` is missing. Later we'll count the words in
    # this string, so this conversion helps us avoid NaNs in the count.
    return ""

posts_with_users_df = posts_with_users_df.assign(
    AboutMeText=posts_with_users_df["AboutMe"].apply(get_html_text)
)

Nice, but `AboutMeText` isn't yet my operationalized feature. For that, I need to count
the number of words in each `AboutMeText` section.

In [8]:
posts_with_users_df = posts_with_users_df.assign(
    AboutMeTextWordCount=posts_with_users_df["AboutMeText"].str.len()
)

Now I turn to counting the prevalence of personal pronouns in each post body. We'll use
`nltk` to tokenize the text, which is probably overkill compared to a regex, but it
should get me to relax about weird whitespace and punctuation trimming and splitting
issues.

In [9]:
import nltk
from nltk.tokenize import word_tokenize

# Download necessary data from nltk
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

personal_pronouns = set(["i", "me", "my", "myself", "mine", "we", "us", "our", "ours"])


def personal_pronoun_rate(text: Optional[str]) -> float:
    """Count the prevalence of personal pronouns in text and return it as a percentage
    of the total words.

    Args:
        text (Optional[str]): Text to process.

    Returns:
        float: What % of words in text are personal pronouns.
    """
    if text in (None, ""):
        return 0

    # Tokenize the text
    words = word_tokenize(text)
    # Filter out stopwords and count personal pronouns
    count = sum(1 for word in words if word.lower() in personal_pronouns)
    return float(count) / len(text)


# Example usage
example_text = "I think this is an amazing movie, and you will love it too."
example_rate = personal_pronoun_rate(example_text)
print(f"Rate of personal pronouns: {example_rate}")

Rate of personal pronouns: 0.01694915254237288


[nltk_data] Downloading package punkt to /Users/kearns/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/kearns/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kearns/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


String-parsing on DataFrames can blow up in your face, so let's be responsible and
estimate completion time with `tqdm`.

In [10]:
from tqdm.notebook import tqdm
tqdm.pandas()

posts_with_users_df = posts_with_users_df.assign(
    BodyPersonalPronounRate=posts_with_users_df["BodyText"].progress_apply(personal_pronoun_rate)
)

  0%|          | 0/57763 [00:00<?, ?it/s]

Finally, we need to go back from the post level of analysis to the user level. We can do
this with a `groupby`, which requires specifying our method for aggregating our
`BodyPersonalPronounRate`s for each user. I'll choose to mean-aggregate, acknowledging
that this comes with its own constraints e.g. outlier robustness. Another option would
be median.

In [11]:
# Note how the user `Id` is now `Id_user` from the merge with posts, where we specified
# a suffix.
user_about_to_pronouns_df = (
    posts_with_users_df[["Id_user", "AboutMeTextWordCount", "BodyPersonalPronounRate"]]
    .groupby("Id_user").agg("mean")  # This aggregates `AboutMeTextWordCount` too, but
                                     # that's fine, since there's only one per `Id_user`
)

In [12]:
user_about_to_pronouns_df.head()

,AboutMeTextWordCount,BodyPersonalPronounRate
Id_user,,
10,133.0,0.008163
100,543.0,0.001856
1000,0.0,0.000000
100024,0.0,0.000583
100025,0.0,0.003563


Now we have a bivariate distribution we could analyze for correlation.

In [13]:
import plotly.express as px

px.scatter(
    user_about_to_pronouns_df,
    x="AboutMeTextWordCount",
    y="BodyPersonalPronounRate"
)

### Evaluator's comments 
 > These will focus on the clarity of the description, the extent to which the description matches both the data and the concept, and the clarity of any visualisation of the operationalised feature. Consider discussing the code in terms of FREE.

^^^

# Question 4. "Ackshually": Partitioning and Aggregating data 

This is a guided analysis of this data which may or may not be directly useful for your specific research question, but will include several interesting steps involving merging and aggregation. This particular analysis will make use of the fact that comments are threaded, meaning that they do not simply have a post associated with them, but potentially also another comment to which they are replying. 

There is now an internet meme about people who like to chime in and correct others, often presumably starting their comment with "Actually, ...". The meme is typified on knowyourmeme with the standard tropes of neck beard, etc... See: https://knowyourmeme.com/memes/ackchyually-actually-guy

In computational social science we can think about structured communication as a series of roles. (See https://www.cmu.edu/joss/content/articles/volume8/Welser/ for an example of how others have operationalised roles). 

Here we will operationalise roles into 4 different signatures. One of these signatures we may associate with the "Ackshually" meme. To identify these signatures we will need to merge the data...with itself. 

Below I describe the four roles we will identify. These should be mutually exclusive. 

1. People who never created any content but have a column in the user_df
2. People who _only_ create a post but never an answer. 
3. People who only create an answer. 
4. People who create both posts and answers. 

Now here's where it gets tricky: 
I want you to separate out role number 3 into:
3.1. People who only create an answer that is a reply to another answer.
3.2. Everyone else who only creates an answer. 

Then let's find out if people in 3.1 are more prone to using the word "actually". As in, these are the people who never ask questions or even provide a useful answer at first, but swoop in to correct someone else's answer. 

Below I explain why this requires you to merge the data in with itself:
- Each post has an "`Id`" column indicating its unique index in the data. 
- Each post has a "`PostTypeId`" to delineate whether it is a question or an answer. 
- Each post has a "`ParentId`" column indicating (if it is a reply) what post it is in reply to. If there is a `'Id'` of 3 for one post, and a `'ParentId`' of 3 for another post, that means that the second post is a reply to the first one. 

You will need to get the PostType for the _parent_ of every answer and merge it in with that answer. Then, if the PostTypeId of the Parent is also 'answer', then this post is a "reply to an answer". It is a one to many merge because you are merging the one PostTypeID of the parent into the many answers. 

Then you have to mark in a separate column the mentions of "actually".  

Finally, you have split the data into the roles above. There are many possible approaches to this task. Mine would be to mark each post as "question", "answer to a question", "answer to another answer". Then I would sum these per user. Then I would find the roles in this aggregated user data set:

Pseudocode: 
- if (count of "question") and (count "a" == 0) and (count "aa" == 0): group_1
- elif (count of "question" > 0) and (count "a" == 0) and (count "aa" == 0): group_2
- elif (count of "question" > 0) and ((count "a" > 0) or (count "aa" > 0)): group_4
- elif (count of "question" == 0) and (count "a" == 0) and (count "aa" > 0): group_3_1
- elif (count of "question" == 0) and ((count "a" > 0) or (count "aa" > 0)): group_3_2

Then once I have these roles, I can merge them back into the posts data, and aggregate the mentions of "actually" by role type. Then we simply report the findings. You encouraged to try a Chi-Square test of independence on the table with groups 2,3.1,3.2, and 4. If the test is not significant it should mean that no group is more likely than another to use this word. The test is somewhat sensitive to cells with less than 5 in there so this may or may not be suitable, but consider some way to assess whether this we can say there are more observed than expected instances of mentioning the word. 

> Please note that this lab is on real data with no prior completion of the lab when setting it. Thus, there might not be any interesting relationship between the groups I describe and the word "actually" or indeed maybe not even any people in one of the groups I will ask you to create. This is not a trick...this is a deductively created exploration of this data set.

In [14]:
# Step 0. Load your data into DataFrames
posts_df = pd.read_feather('data/movies_stack_df_posts.feather')

Per the caveat mentioned about Answers-to-Answers and their non-existence on Stack
Exchange, I am choosing to merge the `movies_stack_df_comments.feather` in with posts,
and consider the class of users who comment on answers as the potential "Ackshually"
user class.

In [15]:
comments_df = pd.read_feather('data/movies_stack_df_comments.feather')

We're going to concatenate posts and comments data together. This requires a bit of
careful "schema coercion" to make sure columns exactly match.

In [16]:
# These are the only columns I care about
columns = ["OwnerUserId", "Id", "PostTypeId", "ParentId", "BodyText"]
posts_df = posts_df[columns]

In [17]:
comments_df = (
    comments_df.rename(columns={
        "Text": "BodyText",
        "UserId": "OwnerUserId",
        "PostId": "ParentId",  # Posts are 'parents' of comments
    })

    # Use a special '-1' post type, which we will later use to ID these with
    # `PostTypeLabel` = "Comment"
    .assign(PostTypeId="-1")

    # Finally, prefix `Id`s with "Comment_" so they don't clash with post IDs
    .assign(Id="Comment_" + comments_df["Id"])
)

# Ensure we have the exact same schema as `posts_df`, so we can concat]
comments_df = comments_df[columns]

In [18]:
posts_and_comments_df = pd.concat([posts_df, comments_df])

In [19]:
# Step 1. Create an "actually" column. 
# This can be True or False depending on whether the text contains the word "actually". 
# Structure this in such a way that you can just as easily ask for a different word
# This means you should probably create a function and use "check for word" as some parameter, then 
# send check_for_word("actually") so later you can check for another word or phrase.

def check_for_word(text: str, word: str) -> bool:
    return word in text

# I'm going to call this `TargetWord` to reflect that words other than "actually" can be
# chosen
posts_and_comments_df["TargetWord"] = (
    posts_and_comments_df["BodyText"].apply(check_for_word, word="actually")
)

In [20]:
# Step 2. Identify which rows refer to questions and which refer to answers 

## For this you can use the post schema: https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede/326361
## To be more direct you will want to filter the data in some manner based on `PostTypeId`
## You might want separate `is_post` and `is_answer` columns or perhaps just filter `PostTypeId` removing other rows

def link_comment_threads(df: pd.DataFrame) -> pd.DataFrame:
    # Add labels for each post type, mapped from PostTypeId
    # This code is copied exactly from the Week01_1_VW_Distributions notebook
    new_labels = {
        '1': "Question",
        '2': "Answer", 
        '3': "Orphaned tag wiki",
        '4': "Tag wiki excerpt",
        '5': "Tag wiki",
        '6': "Moderator nomination",
        '7': "Wiki placeholder",
        '-1': "Comment"  # This is added specially to handle comment data
    }

    df["PostTypeLabel"] = (
        df["PostTypeId"].map(new_labels)
    )

    # Filter for just questions, answers, and comments
    df = df[df["PostTypeLabel"].isin(["Question", "Answer", "Comment"])]

    # Step 3. Identify which answer rows reply to an answer or a question
    # 
    # For this, look to the `ParentId` column. 
    # Now here it can get a bit tricky. You will want to merge in the PostTypeId of the parent's row into the child's row.
    # Then you can filter the dataset into top-level answers and other other answers. 
    df = df.merge(
        df,
        left_on="ParentId",
        right_on="Id",
        how="left",  # Left join preserves Question posts, which have no parents
        suffixes=("", "Parent")  # the right table is the parent, so we'll be interested
                                # in `PostTypeLabelParent`
    )

    # Now, have to drop comments that couldn't find their parents in the `post_df` data.
    # (sorry, little ones...)
    df = df[
        ~(df["PostTypeLabelParent"].isna()) |
        ~(df["PostTypeLabel"] == "Comment")
    ]

    return df

In [21]:
posts_and_comments_df = link_comment_threads(posts_and_comments_df)

In [22]:
def reply_type(label: str, parent_label: str) -> str:
    """Specify the 'reply type' of a post.

    Args:
        label (str): The PostTypeLabel of the child post.
        parent_label (str): The PostTypeLabel of the parent post.

    Raises:
        ValueError: If we aren't expecting this combination of `label` and
            `parent_label`.

    Returns:
        str: One of 'Question', 'AnswerToQuestion', or 'AnswerToAnswer'.
    """
    if label == "Question": return "Question"
    if label == "Answer" and parent_label == "Question": return "AnswerToQuestion"
    if label == "Comment" and parent_label == "Question": return "CommentOnQuestion"
    if label == "Comment" and parent_label == "Answer": return "CommentOnAnswer"
    else: raise ValueError(
        f"Unknown child-parent post combination: {label}, {parent_label}"
    )

def assign_reply_type(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(
        ReplyType=posts_and_comments_df.apply(
            lambda row: reply_type(row["PostTypeLabel"], row["PostTypeLabelParent"]),
            axis="columns"  # Equivalently axis=1, meaning we apply to each DataFrame
                            # row
        )
    )

# Assign the `ReplyType` column
posts_and_comments_df = assign_reply_type(posts_and_comments_df)

In [23]:
# Little sanity check
posts_and_comments_df["ReplyType"].value_counts()

ReplyType
CommentOnAnswer      54142
CommentOnQuestion    46795
AnswerToQuestion     37036
Question             22570
Name: count, dtype: int64

Given that I'm integrating and considering comment data, the user role breakdown will be
slightly different from originally suggested. Here's what I'll do:

1. People who never created any content but have a column in the user_df.
2. People who _only_ create a post but never comment or answer. 
3. People who only create comments or answers.
4. People who create both posts and comments / answers.

With subgroups:

3.1. People who only comment, but don't answer.

3.2. People who answer and may comment.

In [24]:
# Step 4. Create role-specific labels 
# See explaination above

def group_by_reply_type(df: pd.DataFrame) -> pd.DataFrame:
    return (
        df
        .groupby(["OwnerUserId"])
        .agg(
            # Get value counts of the categorical values of `ReplyType` using lambda
            # function aggregations
            Question=("ReplyType", lambda x: (x == "Question").sum()),
            AnswerToQuestion=("ReplyType", lambda x: (x == "AnswerToQuestion").sum()),
            CommentOnQuestion=("ReplyType", lambda x: (x == "CommentOnQuestion").sum()),
            CommentOnAnswer=("ReplyType", lambda x: (x == "CommentOnAnswer").sum()),

            # Get frequencies of target word as counts of each boolean T/F value
            TargetWordPresent=("TargetWord", lambda x: (x == True).sum()),
            TargetWordAbsent=("TargetWord", lambda x: (x == False).sum()),
        )
    )

grouped_user_replies_df = group_by_reply_type(posts_and_comments_df)

In order to create group 1, we need data from the `movies_stack_df_users` table to count
users with no data in either `posts_df` or `comments_df`.

In [25]:
users_df = pd.read_feather('data/movies_stack_df_users.feather')

users_df = users_df.rename(columns={"Id": "OwnerUserId"})
users_df = users_df[["OwnerUserId"]]

In [26]:
all_users_with_replies_df = users_df.merge(
    grouped_user_replies_df,
    on="OwnerUserId",
    how="left",
).fillna(0)  # Empty values mean no post / comment data was found, so fill with 0

In [27]:
all_users_with_replies_df.head()

,OwnerUserId,Question,AnswerToQuestion,CommentOnQuestion,CommentOnAnswer,TargetWordPresent,TargetWordAbsent
0,-1,0.0,0.0,5.0,28.0,0.0,33.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1.0,0.0,1.0,0.0,0.0,2.0


In [28]:
# Step 5. Aggregate the data into the different roles and report on it.
# See explaination above 

def user_role_from_post_activity(
    questions: int,
    answers_to_questions: int,
    comments_on_questions: int,
    comments_on_answers: int,
) -> str:
    if sum(
        [questions, answers_to_questions, comments_on_questions, comments_on_answers]
    ) == 0:
        # No post activity at all
        return "1"

    if (
        questions > 0 and
        sum([answers_to_questions, comments_on_questions, comments_on_answers]) == 0
    ):
        # Only asks questions
        return "2"

    if questions == 0:
        if (
            answers_to_questions == 0 and
            comments_on_questions + comments_on_answers > 0
        ):
            # Only comments, but doesn't directly answer questions
            return "3.1"
        
        if answers_to_questions > 0:
            # Answers questions and may or may not comment
            return "3.2"

    if questions > 0 and sum(
        [answers_to_questions, comments_on_questions, comments_on_answers]
    ) > 0:
        # Posts both questions and answers / comments
        return "4"
    
    raise ValueError(
        "Couldn't categorize user! "
        f"{questions} questions; "
        f"{answers_to_questions} answers; "
        f"{comments_on_questions} comments on questions; "
        f"{comments_on_answers} comments on answers."
    )

def assign_user_roles(df: pd.DataFrame) -> pd.DataFrame:
    # We'll use the same trick to apply the above function across rows
    return df.assign(
        UserRole=all_users_with_replies_df.apply(
            lambda row: user_role_from_post_activity(
                row["Question"],
                row["AnswerToQuestion"],
                row["CommentOnQuestion"],
                row["CommentOnAnswer"],
            ),
            axis="columns"  # Equivalently axis=1, meaning we apply to each DataFrame row
        )
    )

all_users_with_replies_df = assign_user_roles(all_users_with_replies_df)

In [29]:
all_users_with_replies_df.head()

,OwnerUserId,Question,AnswerToQuestion,CommentOnQuestion,CommentOnAnswer,TargetWordPresent,TargetWordAbsent,UserRole
0,-1,0.0,0.0,5.0,28.0,0.0,33.0,3.1
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1
3,3,0.0,0.0,0.0,0.0,0.0,0.0,1
4,4,1.0,0.0,1.0,0.0,0.0,2.0,4


Now we're in a position to aggregate the occurrences of the target word ("actually") by
user role.

In [30]:
actually_by_user_role_df = (
    all_users_with_replies_df
    .groupby("UserRole")[["TargetWordPresent", "TargetWordAbsent"]]
    .agg("sum")
)
actually_by_user_role_df

,TargetWordPresent,TargetWordAbsent
UserRole,,
1,0.0,0.0
2,186.0,3034.0
3.1,304.0,6000.0
3.2,1858.0,24101.0
4,7318.0,112150.0


In [31]:
from scipy import stats

chi2, p, dof, expected = stats.chi2_contingency(
    # Drop user group 1, which definitionally has no post activity—the 0 overall count
    # will mess up the chi-square statistic
    actually_by_user_role_df.drop(index="1")
)

print(f"Chi-squared statistic: {chi2:.3f}")
print(f"p-value: {p:.3f}")
print(f"Degrees of freedom: {dof}")

Chi-squared statistic: 62.878
p-value: 0.000
Degrees of freedom: 3


The chi-squared statistic and p-value suggest a meaningful difference in the use of
"actually" across user groups!

In [32]:
actually_by_user_role_df["TargetWordPct"] = (
    actually_by_user_role_df["TargetWordPresent"] /
    actually_by_user_role_df["TargetWordAbsent"]
)
px.bar(
    actually_by_user_role_df["TargetWordPct"],
    title="Prevalence of 'actually' by User Role type"
)

## Evaluator's comments

> These will focus on the clarity of the code and the plausibility of the result given the code. 

^^^

# Exercise 5. Testing a research hypothesis 

In this part, please reuse your code pipeline in question 4. Except instead of using the word "actually", use a word, phrase, or feature of the body text which you think might credibly differ between these four different classes of users (not five since the one's who do not post would not count). For example, in the Movie SE, one might inquire about the use of the word "cinematic", for example. Posit a hypothesis that suggests there is a significant difference in the use of `[feature x]` between these four classes. Then report the table and the Chi-square results. Please, try to think of an interesting word/feature and then use that, rather than iterate until you find one that is sigificant. However I understand there is some rationale to exploring different possible words. But try to treat this as a deductive task. 

## Answer 5: 

My hypothesis: The punctuation **"!"** will be more prevalently used by user groups 3.1
and 3.2 than other user groups, and particularly in group 3.1.

My rationale: Comments are more likely to be settings to express high emotional arousal
("This is preposterous!") compared with the possibly more dry and formulaic question and
answer interplay on Stack Exchange.

In [33]:
# Answer 5 Here

# Start with the original concatenation of posts and comments
posts_and_comments_df = pd.concat([posts_df, comments_df])

# Collect target word counts for "!"
posts_and_comments_df["TargetWord"] = (
    posts_and_comments_df["BodyText"].apply(check_for_word, word="!")
)

# Link threads of questions, answers, and comments
posts_and_comments_df = link_comment_threads(posts_and_comments_df)

# Assign the `ReplyType` column
posts_and_comments_df = assign_reply_type(posts_and_comments_df)

# Perform the group  by `ReplyType` to get a DataFrame at the user level
grouped_user_replies_df = group_by_reply_type(posts_and_comments_df)

# Again, we have to merge with the users_df to create group 1
all_users_with_replies_df = users_df.merge(
    grouped_user_replies_df,
    on="OwnerUserId",
    how="left",
).fillna(0)  # Empty values mean no post / comment data was found, so fill with 0

# Now assign user roles based on post activity
all_users_with_replies_df = assign_user_roles(all_users_with_replies_df)

# Finally, we should have our comparative frequencies for "!"!
exclam_by_user_role_df = (
    all_users_with_replies_df
    .groupby("UserRole")[["TargetWordPresent", "TargetWordAbsent"]]
    .agg("sum")
)
exclam_by_user_role_df

,TargetWordPresent,TargetWordAbsent
UserRole,,
1,0.0,0.0
2,133.0,3087.0
3.1,345.0,5959.0
3.2,1650.0,24309.0
4,9122.0,110346.0


In [36]:
from scipy import stats

chi2, p, dof, expected = stats.chi2_contingency(
    # Drop user group 1, which definitionally has no post activity—the 0 overall count
    # will mess up the chi-square statistic
    exclam_by_user_role_df.drop(index="1")
)

print(f"Chi-squared statistic: {chi2:.3f}")
print(f"p-value: {p:.3f}")
print(f"Degrees of freedom: {dof}")

Chi-squared statistic: 133.260
p-value: 0.000
Degrees of freedom: 3


Another statistically significant difference between frequencies.

In [37]:
exclam_by_user_role_df["TargetWordPct"] = (
    exclam_by_user_role_df["TargetWordPresent"] /
    exclam_by_user_role_df["TargetWordAbsent"]
)
px.bar(
    exclam_by_user_role_df["TargetWordPct"],
    title="Prevalence of '!' by User Role type"
)

My explanation of the results:

I was incorrect in my hypothesis. There _is_ significant difference in the prevalence of
exclamations across the different user roles, but not among the groups that only answer
or comment. In contrast, it seems that the more engaged the user is with the forum
(answering and commenting, and even more so questioning, answering and commenting), the
more they use exclamations. This makes sense, as we would expect users that engage more
with the actual discussion occurring on Stack Exchange have more of an emotional
investment in the topic.

## Evaluator's comments: 

> These will focus on the plausibility of the relationship given the code, the clarity of the hypothesis, and the effectiveness of the approach to 'just exploring'. 

^^^